In [9]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local').appName('Read_CPU_log_data').getOrCreate()
sc = spark

In [10]:
cpu_log=spark.read.format("csv")\
            .option("header","true")\
            .option("inferschema","true")\
            .option("mode","FAILFAST")\
            .load(r"C:\Users\Lenovo\Downloads\CPU-LOG-DATA\CpuLogData2019-09-20.csv")
cpu_log.show()
cpu_log_sensor = cpu_log.select("DateTime","user_name","keyboard","mouse")
cpu_log_sensor.show()

+-------------------+---------+----------------+-------------+-----------+----------------+----------------------------------------+---------------------------------+-------------------------------+-----------------------+-----------------------+------------------------+-------------------+------------------+------------------+--------------------+----------------------+---------------------+--------------------+--------------------+----------------------+-----------------+----------------+----------------+---------------+----------------+---------------+----------------+----------------------------+--------------------------+----------------------------+--------------------+------------------------+----------------------+-------------------------+--------------------------------------+------------------------------------+---------------------------------------------------+---------------------------------------------------+--------------------+--------------------+--------+------+----

In [11]:
cpu_log_sensor.show()


+-------------------+--------------------+--------+------+
|           DateTime|           user_name|keyboard| mouse|
+-------------------+--------------------+--------+------+
|2019-09-20 09:05:01|sharlawar77@gmail...|    39.0| 210.0|
|2019-09-20 09:10:01|sharlawar77@gmail...|     0.0|   0.0|
|2019-09-20 09:15:02|sharlawar77@gmail...|    99.0| 575.0|
|2019-09-20 09:20:06|sharlawar77@gmail...|     9.0|1657.0|
|2019-09-20 09:25:05|sharlawar77@gmail...|   298.0| 356.0|
|2019-09-20 09:30:05|sharlawar77@gmail...|   170.0| 221.0|
|2019-09-20 09:35:06|sharlawar77@gmail...|   270.0|  48.0|
|2019-09-20 09:40:05|sharlawar77@gmail...|   206.5| 364.0|
|2019-09-20 09:45:05|sharlawar77@gmail...|   248.0| 227.0|
|2019-09-20 09:50:05|sharlawar77@gmail...|   304.5|1638.0|
|2019-09-20 09:55:06|sharlawar77@gmail...|   154.5| 213.0|
|2019-09-20 10:00:01|  iamnzm@outlook.com|    23.0| 208.0|
|2019-09-20 10:00:06|sharlawar77@gmail...|    88.0| 261.0|
|2019-09-20 10:05:01|  iamnzm@outlook.com|     2.0|  65.

find lowest average hours

In [12]:
from pyspark.sql.functions import avg
average_hours_df = cpu_log_sensor.groupBy("user_name").agg(avg("keyboard").alias("avg_keyboard"), avg("mouse").alias("avg_mouse"))
lowest_avg_hours_df = average_hours_df.orderBy("avg_keyboard", "avg_mouse")
lowest_avg_hours_df.show()

+--------------------+------------------+------------------+
|           user_name|      avg_keyboard|         avg_mouse|
+--------------------+------------------+------------------+
|markfernandes66@g...|14.058823529411764| 445.6470588235294|
|  iamnzm@outlook.com|23.754545454545454| 329.9818181818182|
|rahilstar11@gmail...| 27.08823529411765|119.07843137254902|
|damodharn21@gmail...|44.826530612244895|2153.1020408163267|
|sharlawar77@gmail...| 45.97727272727273|             307.0|
|bhagyashrichalke2...|110.02173913043478| 2864.717391304348|
|salinabodale73@gm...|           1343.81|            119.88|
+--------------------+------------------+------------------+



find highest avg hours

In [13]:
average_hours_df = cpu_log_sensor.groupBy("user_name").agg(avg("keyboard").alias("avg_keyboard"), avg("mouse").alias("avg_mouse"))
highest_avg_hours_df = average_hours_df.orderBy("avg_keyboard", "avg_mouse", ascending=False)
highest_avg_hours_df.show()


+--------------------+------------------+------------------+
|           user_name|      avg_keyboard|         avg_mouse|
+--------------------+------------------+------------------+
|salinabodale73@gm...|           1343.81|            119.88|
|bhagyashrichalke2...|110.02173913043478| 2864.717391304348|
|sharlawar77@gmail...| 45.97727272727273|             307.0|
|damodharn21@gmail...|44.826530612244895|2153.1020408163267|
|rahilstar11@gmail...| 27.08823529411765|119.07843137254902|
|  iamnzm@outlook.com|23.754545454545454| 329.9818181818182|
|markfernandes66@g...|14.058823529411764| 445.6470588235294|
+--------------------+------------------+------------------+



                                     idlehours

In [14]:
cpu_log_sensor.createOrReplaceTempView("user_activity")
sql_query = """
    SELECT user_name, COUNT(*) AS idle_hours
    FROM user_activity
    WHERE keyboard = 0 AND mouse = 0
    GROUP BY user_name
    ORDER BY idle_hours DESC
"""
result_df = spark.sql(sql_query)
result_df.show()

+--------------------+----------+
|           user_name|idle_hours|
+--------------------+----------+
|sharlawar77@gmail...|        38|
|rahilstar11@gmail...|        25|
|  iamnzm@outlook.com|        21|
|damodharn21@gmail...|        15|
|salinabodale73@gm...|         9|
|bhagyashrichalke2...|         5|
|markfernandes66@g...|         3|
+--------------------+----------+



late comers

In [18]:
spark.sql("""select user_name, late_comers, dense_rank() over (ORDER BY late_comers desc) as rank 
          from (select user_name, count(*) as late_comers from user_activity where DateTime > "2019-09-17 08:31:00" 
          group by user_name) as counts order by late_comers desc""").show()

+--------------------+-----------+----+
|           user_name|late_comers|rank|
+--------------------+-----------+----+
|sharlawar77@gmail...|         66|   1|
|  iamnzm@outlook.com|         55|   2|
|rahilstar11@gmail...|         51|   3|
|markfernandes66@g...|         51|   3|
|salinabodale73@gm...|         50|   4|
|damodharn21@gmail...|         49|   5|
|bhagyashrichalke2...|         46|   6|
+--------------------+-----------+----+



upload it output to sql server